In [1]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

In [2]:
#定义差分进化灰狼优化函数
np.random.seed(2)
def DEWO_SVM(X_train, X_test, y_train, y_test, pop_size, max_iter):

    # 设置SVR模型的参数范围
    C_range = np.logspace(-5, 5, 20)
    gamma_range = np.logspace(-5, 5, 20)

    # 初始化狼群的位置
    positions = np.random.rand(pop_size, 2) * 10 - 5
    best_position = None
    best_fitness = float('-inf')

    # 开始迭代
    for i in range(max_iter):
        # 计算每只狼的适应度
        fitness = np.zeros(pop_size)
        for j in range(pop_size):
            C = C_range[int(round(positions[j, 0]))]
            gamma = gamma_range[int(round(positions[j, 1]))]
            clf = svm.SVR(kernel='rbf', C=C, gamma=gamma)
            clf.fit(X_train, y_train)
            fitness[j] = clf.score(X_test, y_test)

            # 更新全局最优解
            if fitness[j] > best_fitness:
                best_fitness = fitness[j]
                best_position = positions[j]

        # 对狼群进行排序
        rank = np.argsort(-fitness)
        alpha = positions[rank[0]]
        beta = positions[rank[1]]
        delta = positions[rank[2]]

        # 更新每个个体的位置
        for j in range(pop_size):
            # 差分变异
            mutation = beta + 0.5 * (delta - beta)
            # 杂交
            crossover = np.random.rand(1, 2)
            mask = crossover > 0.5
            intermediate = mask * mutation + (1 - mask) * positions[j]

            # 更新个体位置
            if np.random.rand() < 0.5:
                positions[j] = intermediate
            else:
                positions[j] = alpha + np.random.rand() * (best_position - positions[j])

    # 使用最优参数建立SVR模型
    C_star = C_range[int(round(best_position[0]))]
    gamma_star = gamma_range[int(round(best_position[1]))]
    svr = svm.SVR(kernel='rbf', C=C_star, gamma=gamma_star)
    svr.fit(X_train, y_train)

    # 在测试集上进行预测
    y_pred = svr.predict(X_test)

    # 计算预测结果误差
    mse = np.mean((y_test - y_pred) ** 2)
    return mse, C_star, gamma_star

In [3]:
#导入数据集
url = r'F:\研究生\ML+ESE\碳排放数据\山东省1997-2019carbon原始数据.xlsx'
df = pd.read_excel(url)
scaler=StandardScaler()  #标准化
df_sc= scaler.fit_transform(df)  #对数据先拟合fit，找到均值、方差、最大值最小值等；然后进行转换，实现数据的标准化、归一化
df_sc = pd.DataFrame(df_sc, columns=df.columns)
y = df_sc['carbon（y）']
X = df_sc.drop('carbon（y）', axis=1) # becareful inplace= False
# 将数据集分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [4]:
#调用差分进化灰狼优化函数
# 设置种群规模和最大迭代次数
pop_size = 50
max_iter = 100
mse, C_star, gamma_star = DEWO_SVM(X_train, X_test, y_train, y_test, pop_size, max_iter)
print('最优参数C: %.4f' % C_star)
print('最优参数gamma: %.4f' % gamma_star)
print('预测结果误差: %.4f' % mse)

最优参数C: 233.5721
最优参数gamma: 0.0144
预测结果误差: 0.0083
